# Porto Seguro Safe Driver Prediction - Data Processing and EDA

This notebook provides a starting point for processing and analyzing the Porto Seguro Safe Driver Prediction dataset.

## Sections:
1. **Setup and Data Extraction** - Import libraries and extract the dataset from the zip file
2. **Data Loading** - Read the extracted CSV files and display their structure
3. **Exploratory Data Analysis** - Perform basic EDA including summary statistics and visualizations
4. **Initial Insights** - Document key findings from the exploration

## 1. Setup and Data Extraction

First, we'll import the necessary libraries and extract the dataset from the zip file.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import warnings

# Configure visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("Libraries imported successfully!")

In [ ]:
# Define file paths
zip_file_path = 'data/porto-seguro-safe-driver-prediction.zip'
extraction_path = 'data/'

# Check if zip file exists
if os.path.exists(zip_file_path):
    print(f"Zip file found: {zip_file_path}")
    print(f"File size: {os.path.getsize(zip_file_path) / (1024**2):.2f} MB")
else:
    print(f"Error: Zip file not found at {zip_file_path}")

In [ ]:
# Extract the zip file
print("Extracting zip file...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # List contents of the zip file
    file_list = zip_ref.namelist()
    print(f"\nFiles in archive: {file_list}")
    
    # Extract all files
    zip_ref.extractall(extraction_path)
    print(f"\nFiles extracted to: {extraction_path}")

# Verify extraction
extracted_files = [f for f in os.listdir(extraction_path) if f.endswith('.csv')]
print(f"\nExtracted CSV files: {extracted_files}")

## 2. Data Loading and Structure

Now we'll load the extracted data files and examine their structure.

In [ ]:
# Load the training dataset
print("Loading training data...")
train_df = pd.read_csv(os.path.join(extraction_path, 'train.csv'))
print(f"Training data loaded successfully!")
print(f"Shape: {train_df.shape}")
print(f"Rows: {train_df.shape[0]:,}, Columns: {train_df.shape[1]}")

In [ ]:
# Display the first few rows
print("First 5 rows of the training dataset:\n")
train_df.head()

In [ ]:
# Display column information
print("Dataset Information:\n")
train_df.info()

In [ ]:
# Display column names and their data types
print("Column Names and Data Types:\n")
column_info = pd.DataFrame({
    'Column': train_df.columns,
    'Data Type': train_df.dtypes.values,
    'Non-Null Count': train_df.count().values,
    'Null Count': train_df.isnull().sum().values
})
column_info

In [ ]:
# Load test dataset (for reference)
print("Loading test data...")
test_df = pd.read_csv(os.path.join(extraction_path, 'test.csv'))
print(f"Test data loaded successfully!")
print(f"Shape: {test_df.shape}")
print(f"Rows: {test_df.shape[0]:,}, Columns: {test_df.shape[1]}")

## 3. Exploratory Data Analysis (EDA)

Let's perform comprehensive exploratory data analysis to understand the dataset better.

### 3.1 Summary Statistics

In [ ]:
# Generate summary statistics for all numerical columns
print("Summary Statistics:\n")
train_df.describe()

In [ ]:
# Check the target variable distribution
print("Target Variable Distribution:\n")
print(train_df['target'].value_counts())
print(f"\nTarget Distribution (%):\n{train_df['target'].value_counts(normalize=True) * 100}")

### 3.2 Missing Values Analysis

In [ ]:
# Check for missing values
missing_values = train_df.isnull().sum()
missing_percentage = (missing_values / len(train_df)) * 100

missing_df = pd.DataFrame({
    'Column': train_df.columns,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
})

# Show only columns with missing values
missing_df_filtered = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df_filtered) > 0:
    print("Columns with Missing Values:\n")
    print(missing_df_filtered)
else:
    print("No missing values found in the dataset!")

### 3.3 Feature Analysis

The dataset contains different types of features:
- **Binary features** (suffix: _bin)
- **Categorical features** (suffix: _cat)
- **Continuous/Ordinal features** (no suffix or other suffixes)

In [ ]:
# Categorize features by type
binary_features = [col for col in train_df.columns if '_bin' in col]
categorical_features = [col for col in train_df.columns if '_cat' in col]
continuous_features = [col for col in train_df.columns 
                       if col not in binary_features + categorical_features + ['id', 'target']]

print(f"Binary features ({len(binary_features)}): {binary_features[:5]}...")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features[:5]}...")
print(f"\nContinuous/Ordinal features ({len(continuous_features)}): {continuous_features[:5]}...")

### 3.4 Visualizations

In [ ]:
# Plot target variable distribution
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
train_df['target'].value_counts().plot(kind='bar', ax=ax[0], color=['skyblue', 'salmon'])
ax[0].set_title('Target Variable Distribution (Count)', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Target', fontsize=12)
ax[0].set_ylabel('Count', fontsize=12)
ax[0].set_xticklabels(['No Claim (0)', 'Claim (1)'], rotation=0)

# Pie chart
train_df['target'].value_counts().plot(kind='pie', ax=ax[1], autopct='%1.1f%%', 
                                        colors=['skyblue', 'salmon'], startangle=90)
ax[1].set_title('Target Variable Distribution (%)', fontsize=14, fontweight='bold')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

print(f"\nNote: The dataset is imbalanced with {(train_df['target']==0).sum()} non-claimants and {(train_df['target']==1).sum()} claimants.")

In [ ]:
# Distribution of some continuous features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

# Select first 6 continuous features for visualization
features_to_plot = continuous_features[:6]

for idx, feature in enumerate(features_to_plot):
    axes[idx].hist(train_df[feature].dropna(), bins=50, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {feature}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(feature, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis - select a subset of features for better visualization
# We'll look at correlations among continuous features
features_for_corr = continuous_features[:10]  # First 10 continuous features
correlation_matrix = train_df[features_for_corr + ['target']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix (Sample Features)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Feature correlation with target variable
target_correlations = train_df.corr()['target'].sort_values(ascending=False)

# Plot top 15 positive and negative correlations
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Top positive correlations
top_positive = target_correlations[1:16]  # Exclude target itself
top_positive.plot(kind='barh', ax=axes[0], color='green', alpha=0.7)
axes[0].set_title('Top 15 Positive Correlations with Target', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Correlation Coefficient', fontsize=10)

# Top negative correlations
top_negative = target_correlations[-15:]
top_negative.plot(kind='barh', ax=axes[1], color='red', alpha=0.7)
axes[1].set_title('Top 15 Negative Correlations with Target', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Correlation Coefficient', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Analyze binary features
if len(binary_features) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()
    
    # Plot first 6 binary features
    for idx, feature in enumerate(binary_features[:6]):
        train_df[feature].value_counts().plot(kind='bar', ax=axes[idx], color='teal', alpha=0.7)
        axes[idx].set_title(f'{feature} Distribution', fontsize=12, fontweight='bold')
        axes[idx].set_xlabel('Value', fontsize=10)
        axes[idx].set_ylabel('Count', fontsize=10)
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=0)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Analyze categorical features - show unique values
if len(categorical_features) > 0:
    print("Categorical Features - Unique Value Counts:\n")
    cat_summary = pd.DataFrame({
        'Feature': categorical_features[:10],  # First 10 categorical features
        'Unique Values': [train_df[col].nunique() for col in categorical_features[:10]]
    })
    print(cat_summary)
    
    # Plot unique value counts
    plt.figure(figsize=(14, 6))
    cat_summary.plot(x='Feature', y='Unique Values', kind='bar', 
                     color='purple', alpha=0.7, legend=False)
    plt.title('Number of Unique Values in Categorical Features', fontsize=14, fontweight='bold')
    plt.xlabel('Feature', fontsize=12)
    plt.ylabel('Unique Value Count', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 4. Initial Insights

Based on the exploratory data analysis, here are some key observations:

### Dataset Characteristics:
- The dataset contains insurance claim data with multiple feature types (binary, categorical, and continuous)
- Features are anonymized with generic names (ps_)
- The target variable indicates whether a claim was filed (1) or not (0)

### Key Findings:
1. **Class Imbalance**: The target variable shows significant class imbalance - claims are much less frequent than non-claims
2. **Feature Types**: The dataset contains:
   - Binary features (suffix: _bin)
   - Categorical features (suffix: _cat)
   - Continuous/ordinal features
3. **Missing Values**: Check if any features have missing values that need handling
4. **Feature Correlations**: Some features show correlation with the target variable

### Next Steps:
- Handle missing values appropriately
- Address class imbalance using techniques like SMOTE, class weights, or sampling
- Feature engineering and selection
- Model selection and training
- Evaluation using appropriate metrics for imbalanced classification

## Summary

This notebook provided:
1. ✅ Extraction of the Porto Seguro dataset from the zip file
2. ✅ Loading and displaying the structure of the data
3. ✅ Comprehensive exploratory data analysis including:
   - Summary statistics
   - Missing value analysis
   - Feature type categorization
   - Distribution visualizations
   - Correlation analysis
4. ✅ Documentation and insights for each analysis step

This notebook serves as a foundation for further analysis and modeling work on the Porto Seguro Safe Driver Prediction dataset.